# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data Description](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction <a name="introduction"></a>
### 1.1. Background
With population of more than 300,000 Howard County, Maryland, USA is considered as one of the best places to live in USA. Its close proximity to Baltimore and Washington, DC as well as having some of the best schools in country makes it very attractive place of residence for mid to upper income families. Because of this, Howard County is very attractive place for small business such as coffee shops, bakeries, and restaurants. This also makes entry to the market very competitive. For someone, for example opening a bakery, choosing the right location for the shop can make or break the business.

### 1.2. Problem
How someone who wants to open for example a bakery should choose the right location? One would argue that close proximity to the exits from the interstate roads might be a good choice since majority of inhabitants of Howard County is using them for daily commute to their work, so it will make business position in such a location very convenient to stop by. However, Covid-19 and work from home phenomenon dramatically changed people’s daily routines, so location near interstate exit may not be a good choice. It will be useful to conduct geo-location analysis to possibly find pattern and understand which location might be more popular than other. 

### 1.3. Interest
For purpose of this exercise, I’ll will do the analysis and try to predict the best spot to open a bakery. Therefore, this research might be of interest to somebody who wants to open a bakery and needs to know which location might be the best to open such a business.

## 2. Data Description <a name="data"></a>

For the analysis I’ll use **Foursquare API** to get most popular venues in Howard County. Also, I’ll check the location of current bakeries and bakeries within grocery stores. 

While some bakeries within grocery stores are listed on Foursquare, Giant Food’s bakery is not. There is no public data available with this information, so I’ll use **Google Maps** to locate such groceries. I’ll also add grocery stores with no bakeries to our dataset, since opening a new bakery near such a business may not be a bad choice. 

To locate county divisions and get their longitude and latitude I’ll use the **Howard County Zip Codes** from [ciclt.net](http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=md&StName=Maryland&StFIPS=24&FIPS=24027) and [OpenDataSoft](https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/?refine.state=MD).

In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         

brotlipy-0.7.0       | 341 KB    | ##################################### | 100% 
backcall-0.2.0       | 13 KB     | ##################################### | 100% 
openjpeg-2.4.0       | 525 KB    | ##################################### | 100% 
pytz-2021.1          | 239 KB    | ##################################### | 100% 
multidict-5.1.0      | 67 KB     | ##################################### | 100% 
importlib-metadata-3 | 23 KB     | ##################################### | 100% 
libzopfli-1.0.3      | 164 KB    | ##################################### | 100% 
pillow-8.1.1         | 668 KB    | ##################################### | 100% 
pyjwt-2.0.1          | 17 KB     | ##################################### | 100% 
biopython-1.78       | 2.6 MB    | ##################################### | 100% 
aiohttp-3.7.4        | 632 KB    | ##################################### | 100% 
libcblas-3.9.0       | 11 KB     | ##################################### | 100% 
_openmp_mutex-4.5    | 5 KB 

nss-3.62             | 2.1 MB    | ##################################### | 100% 
bokeh-2.3.0          | 8.3 MB    | ##################################### | 100% 
traitlets-5.0.5      | 81 KB     | ##################################### | 100% 
oauthlib-3.0.1       | 82 KB     | ##################################### | 100% 
lerc-2.2.1           | 213 KB    | ##################################### | 100% 
libxgboost-1.3.0     | 3.3 MB    | ##################################### | 100% 
xorg-libxau-1.0.9    | 13 KB     | ##################################### | 100% 
bleach-3.3.0         | 111 KB    | ##################################### | 100% 
importlib_metadata-3 | 3 KB      | ##################################### | 100% 
webencodings-0.5.1   | 12 KB     | ##################################### | 100% 
libstdcxx-ng-9.3.0   | 4.0 MB    | ##################################### | 100% 
mistune-0.8.4        | 54 KB     | ##################################### | 100% 
bzip2-1.0.8          | 484 K

plotly-4.14.3        | 5.9 MB    | ##################################### | 100% 
liblapack-3.9.0      | 11 KB     | ##################################### | 100% 
xlsxwriter-1.3.7     | 105 KB    | ##################################### | 100% 
nbformat-5.1.2       | 66 KB     | ##################################### | 100% 
libsodium-1.0.18     | 366 KB    | ##################################### | 100% 
pyyaml-5.4.1         | 189 KB    | ##################################### | 100% 
imagecodecs-2021.1.2 | 6.7 MB    | ##################################### | 100% 
gettext-0.19.8.1     | 3.6 MB    | ##################################### | 100% 
pluggy-0.13.1        | 29 KB     | ##################################### | 100% 
krb5-1.17.2          | 1.4 MB    | ##################################### | 100% 
decorator-4.4.2      | 11 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
pymssql-2.1.5        | 236 K

done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


In [13]:
html_doc = requests.get('http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=md&StName=Maryland&StFIPS=24&FIPS=24027')
try:
    html_doc.raise_for_status()
except Exception as exc:
    print(f'There was a problem: {exc}')
    
print('Ready')

soup = BeautifulSoup(html_doc.text, 'html.parser')

# rows = soup.select('table[border="3"] > tbody > tr')

rows = soup.select('table[border="3"] > tr')

df_rows = []

for i in range(len(rows)):
    row = rows[i].find_all(['th', 'td'])
    df_row =[]
    for td in row:
        try:
            df_row.append(td.text.replace('\n', ''))
        except:
            continue
    if len(df_row) > 0:
        df_rows.append(df_row)
        
df_zip_codes = pd.DataFrame(df_rows[1:], columns=df_rows[0])
df_zip_codes

Ready


,Zip Code,City,County
0,20701,Annapolis Junction,Howard County
1,20723,Scaggsville,Howard County
2,20723,Laurel,Howard County
3,20759,Fulton,Howard County
4,20763,Savage,Howard County
5,20777,Highland,Howard County
6,20794,Jessup,Howard County
7,21029,Clarksville,Howard County
8,21036,Dayton,Howard County
9,21041,Ellicott City,Howard County


In [14]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c2efb76b4b6b42258510492597901f5f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AOJEvaU-d27j4AB4UfJtixLXfXc4gXkWix7rUb8xteW_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c2efb76b4b6b42258510492597901f5f.get_object(Bucket='datasciencecourseracapstone-donotdelete-pr-woppyhzlezhae6',Key='us-zip-code-latitude-and-longitude.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geopoint = pd.read_csv(body)
geopoint.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,21903,Perryville,MD,39.571003,-76.05077,-5,1,"39.571003, -76.05077"
1,20778,West River,MD,38.832265,-76.55604,-5,1,"38.832265, -76.55604"
2,21234,Parkville,MD,39.385006,-76.54177,-5,1,"39.385006, -76.54177"
3,21036,Dayton,MD,39.243670,-76.99782,-5,1,"39.24367, -76.99782"
4,21237,Rosedale,MD,39.332240,-76.50365,-5,1,"39.33224, -76.50365"


In [15]:
geopoint.rename(columns={"Zip": "Zip Code"}, inplace=True)
geopoint['Zip Code'] = geopoint['Zip Code'].map(str)

In [16]:
for i in df_zip_codes.index:
    j = i + 1
    if j <= df_zip_codes.index.stop:
        for j in range(j, df_zip_codes.index.stop):
            if df_zip_codes.iloc[i]['Zip Code'] == df_zip_codes.iloc[j]['Zip Code']:
                icity = df_zip_codes.iloc[i]['City']
                jcity = df_zip_codes.iloc[j]['City']
                df_zip_codes.iloc[i]['City'] = f'{icity}, {jcity}'
                df_zip_codes.iloc[j]['City'] = np.nan
                df_zip_codes.iloc[j]['Zip Code'] = np.nan
            
df_zip_codes.dropna(inplace=True)
df_zip_codes.reset_index(inplace=True, drop = True)
df_zip_codes

,Zip Code,City,County
0,20701,Annapolis Junction,Howard County
1,20723,"Scaggsville, Laurel",Howard County
2,20759,Fulton,Howard County
3,20763,Savage,Howard County
4,20777,Highland,Howard County
5,20794,Jessup,Howard County
6,21029,Clarksville,Howard County
7,21036,Dayton,Howard County
8,21041,Ellicott City,Howard County
9,21042,Ellicott City,Howard County


In [17]:
#df.groupby(by=df.columns, axis=1).sum()

#test = df_zip_codes.groupby(by=df_zip_codes['Zip Code'])
#result = grouped.agg(test)

#df_zip_codes.duplicated(subset=['Zip Code'])

df_zip_codes = pd.merge(df_zip_codes, geopoint, on='Zip Code')
df_zip_codes

,Zip Code,City_x,County,City_y,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,20701,Annapolis Junction,Howard County,Annapolis Junction,MD,39.127708,-76.792300,-5,1,"39.127708, -76.7923"
1,20723,"Scaggsville, Laurel",Howard County,Laurel,MD,39.134062,-76.857900,-5,1,"39.134062, -76.8579"
2,20759,Fulton,Howard County,Fulton,MD,39.147257,-76.937990,-5,1,"39.147257, -76.93799"
3,20763,Savage,Howard County,Savage,MD,39.137307,-76.823350,-5,1,"39.137307, -76.82335"
4,20777,Highland,Howard County,Highland,MD,39.173104,-76.957790,-5,1,"39.173104, -76.95779"
5,20794,Jessup,Howard County,Jessup,MD,39.144807,-76.787670,-5,1,"39.144807, -76.78767"
6,21029,Clarksville,Howard County,Clarksville,MD,39.198816,-76.936700,-5,1,"39.198816, -76.9367"
7,21036,Dayton,Howard County,Dayton,MD,39.243670,-76.997820,-5,1,"39.24367, -76.99782"
8,21041,Ellicott City,Howard County,Ellicott City,MD,39.236405,-76.941902,-5,1,"39.236405, -76.941902"
9,21042,Ellicott City,Howard County,Ellicott City,MD,39.276305,-76.860820,-5,1,"39.276305, -76.86082"


In [18]:
df_zip_codes.drop(columns=['City_y', 'State', 'Timezone', 'Daylight savings time flag', 'geopoint'], inplace=True)
df_zip_codes.rename(columns={'City_x': 'City'}, inplace=True)
df_zip_codes

,Zip Code,City,County,Latitude,Longitude
0,20701,Annapolis Junction,Howard County,39.127708,-76.792300
1,20723,"Scaggsville, Laurel",Howard County,39.134062,-76.857900
2,20759,Fulton,Howard County,39.147257,-76.937990
3,20763,Savage,Howard County,39.137307,-76.823350
4,20777,Highland,Howard County,39.173104,-76.957790
5,20794,Jessup,Howard County,39.144807,-76.787670
6,21029,Clarksville,Howard County,39.198816,-76.936700
7,21036,Dayton,Howard County,39.243670,-76.997820
8,21041,Ellicott City,Howard County,39.236405,-76.941902
9,21042,Ellicott City,Howard County,39.276305,-76.860820


In [19]:
address = 'Howard County, Maryland'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Howard County, Maryland are {latitude}, {longitude}.')

The geograpical coordinate of Howard County, Maryland are 39.2305289, -76.9166241.


In [23]:
map_howard_county = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city, zip_code in zip(df_zip_codes['Latitude'], df_zip_codes['Longitude'], df_zip_codes['City'], df_zip_codes['Zip Code']):
    label = f'{zip_code} {city}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_howard_county)  
    
map_howard_county